In [2]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# Chage cwd
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\MLB')
print(f'CWD: \n{os.getcwd()}')

CWD: 
c:\Users\tyler\OneDrive\Documents\Python\MLB


In [124]:
# Load schedules
schedule = pd.DataFrame()
for season in range(2022-5, 2022):
    schedule = schedule.append(pd.read_csv(f'backend/data/schedules/{season}.csv'), ignore_index=True)

schedule['date'] = pd.to_datetime(schedule['date'])
schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10815 entries, 0 to 10814
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     10815 non-null  datetime64[ns]
 1   visitor  10815 non-null  object        
 2   home     10815 non-null  object        
 3   season   10815 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 338.1+ KB


In [125]:
# Load scores
scores = pd.read_csv('backend/data/scores/boxscore.csv')
scores['date'] = pd.to_datetime(scores['date'])

scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22828 entries, 0 to 22827
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     22828 non-null  datetime64[ns]
 1   visitor  22828 non-null  object        
 2   home     22828 non-null  object        
 3   team     22828 non-null  object        
 4   1        22828 non-null  int64         
 5   2        22828 non-null  int64         
 6   3        22828 non-null  int64         
 7   4        22828 non-null  int64         
 8   5        22828 non-null  object        
 9   6        22820 non-null  object        
 10  7        22802 non-null  object        
 11  8        22378 non-null  object        
 12  9        22322 non-null  object        
 13  R        22828 non-null  int64         
 14  H        22828 non-null  int64         
 15  E        22828 non-null  int64         
 16  10       1872 non-null   float64       
 17  11       850 non-null    float6

In [126]:
# Load batting totals
batting_totals = pd.read_csv('backend/data/batting/totals.csv')
batting_totals['date'] = pd.to_datetime(batting_totals['date'])
batting_totals['wpa-'] = batting_totals['wpa-'].apply(lambda x: float(x.strip("%")))
batting_totals['cwpa'] = batting_totals['cwpa'].apply(lambda x: float(x.strip("%")))
batting_totals = batting_totals.fillna(0)
batting_totals = batting_totals[batting_totals['ab'] != 0]

batting_totals.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22828 entries, 0 to 22827
Data columns (total 46 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     22828 non-null  datetime64[ns]
 1   visitor  22828 non-null  object        
 2   home     22828 non-null  object        
 3   team     22828 non-null  object        
 4   ab       22828 non-null  int64         
 5   r        22828 non-null  int64         
 6   h        22828 non-null  int64         
 7   rbi      22828 non-null  int64         
 8   bb       22828 non-null  int64         
 9   so       22828 non-null  int64         
 10  pa       22828 non-null  int64         
 11  ba       22828 non-null  float64       
 12  obp      22828 non-null  float64       
 13  slg      22828 non-null  float64       
 14  ops      22828 non-null  float64       
 15  pit      22828 non-null  int64         
 16  str      22828 non-null  int64         
 17  wpa      22828 non-null  float6

In [128]:
# Load pitching totals
pitching_totals = pd.read_csv('backend/data/pitching/totals.csv')
pitching_totals['date'] = pd.to_datetime(pitching_totals['date'])
pitching_totals['cwpa'] = pitching_totals['cwpa'].apply(lambda x: float(x.strip('%')))

pitching_totals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22828 entries, 0 to 22827
Data columns (total 30 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     22828 non-null  datetime64[ns]
 1   visitor  22828 non-null  object        
 2   home     22828 non-null  object        
 3   team     22828 non-null  object        
 4   ip       22828 non-null  float64       
 5   h        22828 non-null  int64         
 6   r        22828 non-null  int64         
 7   er       22828 non-null  int64         
 8   bb       22828 non-null  int64         
 9   so       22828 non-null  int64         
 10  hr       22828 non-null  int64         
 11  era      22828 non-null  float64       
 12  bf       22828 non-null  int64         
 13  pit      22828 non-null  int64         
 14  str      22828 non-null  int64         
 15  ctct     22828 non-null  int64         
 16  sts      22828 non-null  int64         
 17  stl      22828 non-null  int64 

In [6]:
# Load pitching details
pitching_details = pd.read_csv('backend/data/pitching/details.csv')
pitching_details['date'] = pd.to_datetime(pitching_details['date'])
pitching_details['cwpa'] = pitching_details['cwpa'].apply(lambda x: float(x.strip('%')))

pitching_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100345 entries, 0 to 100344
Data columns (total 31 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     100345 non-null  datetime64[ns]
 1   visitor  100345 non-null  object        
 2   home     100345 non-null  object        
 3   team     100345 non-null  object        
 4   player   100345 non-null  object        
 5   ip       100345 non-null  float64       
 6   h        100345 non-null  int64         
 7   r        100345 non-null  int64         
 8   er       100345 non-null  int64         
 9   bb       100345 non-null  int64         
 10  so       100345 non-null  int64         
 11  hr       100345 non-null  int64         
 12  era      100306 non-null  float64       
 13  bf       100345 non-null  int64         
 14  pit      100319 non-null  float64       
 15  str      100319 non-null  float64       
 16  ctct     100240 non-null  float64       
 17  sts      1

In [11]:
starters = pd.DataFrame()
bullpen = pd.DataFrame()

for team in pitching_details['team'].unique():
    print(team)
    for season in pitching_details['date'].dt.year.unique():
        print(season)
        team_schedule = pitching_details[
            (pitching_details['team'] == team) &
            (pitching_details['date'].dt.year == season)
        ]
        for date in team_schedule['date'].unique():
            game = team_schedule[team_schedule['date'] == date].copy()
            starters = starters.append(game.iloc[0, :], ignore_index=True)
            bullpen = bullpen.append(game.iloc[1:, :], ignore_index=True)



San Francisco Giants
2017
2018
2019
2020
2022
2021
Arizona Diamondbacks
2017
2018
2019
2020
2022
2021
Chicago Cubs
2017
2018
2019
2020
2022
2021
St. Louis Cardinals
2017
2018
2019
2020
2022
2021
New York Yankees
2017
2018
2019
2020
2022
2021
Tampa Bay Rays
2017
2018
2019
2020
2022
2021
Toronto Blue Jays
2017
2018
2019
2020
2022
2021
Baltimore Orioles
2017
2018
2019
2020
2022
2021
Pittsburgh Pirates
2017
2018
2019
2020
2022
2021
Boston Red Sox
2017
2018
2019
2020
2022
2021
Philadelphia Phillies
2017
2018
2019
2020
2022
2021
Cincinnati Reds
2017
2018
2019
2020
2022
2021
Seattle Mariners
2017
2018
2019
2020
2022
2021
Houston Astros
2017
2018
2019
2020
2022
2021
San Diego Padres
2017
2018
2019
2020
2022
2021
Los Angeles Dodgers
2017
2018
2019
2020
2022
2021
Colorado Rockies
2017
2018
2019
2020
2022
2021
Milwaukee Brewers
2017
2018
2019
2020
2022
2021
Kansas City Royals
2017
2018
2019
2020
2022
2021
Minnesota Twins
2017
2018
2019
2020
2022
2021
Atlanta Braves
2017
2018
2019
2020
2022
2021
N

In [12]:
starters.to_csv('backend/data/pitching/starters.csv', index=False)
bullpen.groupby(
    ['date', 'visitor', 'home', 'team']
).sum(
).to_csv('backend/data/pitching/bullpen.csv', index=False)